# Galaxy Selector MOC

**Compared to the "Simple Galaxy Selector, this works with Multi-Ordering-Sky-Maps**

***NOTE (this note may be wrong, just copied from old notebook)***: Before anything can happen, download the GW event localization map. Open the terminal and type:
"curl -O https://gracedb.ligo.org/api/superevents/sid/files/bayestar.fits,0"
with sid = superevent ID (could be incorporated into this notebook)

Select the galaxies to observe by WWFI in this simple manner:
- get the LIGO event data
- read the LIGO event data into the notebook, extract some information
- read the DESI database into the notebook, "clean" the data
- get the 90% credible regions for the event, add them as a column to the data
- only keep data within the 90% region
- calculate all the luminosities (& more)
- rank them by luminosity (for now just print the TARGETID), 3D/2D localization and with a luminosity-distance 2D dependant counterpart likelihood

Sources: 
- https://iopscience.iop.org/article/10.3847/0067-0049/226/1/10
- https://emfollow.docs.ligo.org/userguide/tutorial/skymaps.html
- https://emfollow.docs.ligo.org/userguide/tutorial/multiorder_skymaps.html
- https://arxiv.org/pdf/1710.05452.pdf

In [15]:
import numpy as np
from scipy.stats import norm
import scipy as sc

from astropy.io import fits
from astropy.table import Table, hstack
from astropy import table

import healpy as hp
import astropy_healpix as ah

from astropy.table import QTable

import psycopg2

from astropy.cosmology import Planck18
from astropy.coordinates import Distance
from astropy import units as u

import astropy.constants as asc

import matplotlib.pyplot as plt

Specifiy the path where the data is located and then load the data (this loads everything, just have a look)

In [4]:
path = "/Users/juliusgassert/Desktop/Studium/LMU/Physik/BA/Code/bayestar.multiorder.fits"
skymap = QTable.read(path)

skymap[:5]

UNIQ,PROBDENSITY,DISTMU,DISTSIGMA,DISTNORM
,1 / sr,Mpc,Mpc,1 / Mpc2
int64,float64,float64,float64,float64
1058,9.762953888490556e-44,17.34517262809015,0.9236687293542615,0.003314461041632539
1059,1.0018783137648462e-44,18.63995767930237,1.0010655001928837,0.002869850930785714
1061,4.956280171074477e-42,21.827776684973404,3.682410269563899,0.002040766430201995
1062,1.1431944234645642e-45,20.051306236593103,1.1270420983387233,0.0024793894203571994
1063,3.746441270992535e-46,21.26344968099752,2.242205858104221,0.0021874094575685733


### Get basic values from GW event

We now extract some basic information  from the healpix data:
- ipix_max: pixel with highest likelihood
- npix: total number of pixels
- ra, dec: ra and dec of the pixel with the highest likelihood

Then we open the full fits file and extract some more basic info from the header:
- dist_mean: mean distance of the GW event
- dist_std: error of the distance

In [6]:
# most probable sky location
i = np.argmax(skymap["PROBDENSITY"])
uniq = skymap[i]["UNIQ"]
print("UNIQ ID of highest prob sky localization: ", uniq)

UNIQ ID of highest prob sky localization:  42318942


In [10]:
# calculate the most probable pixel, convert to RA and dec
level_max, ipix_max = ah.uniq_to_level_ipix(uniq)
nside = ah.level_to_nside(level_max)
ra, dec = ah.healpix_to_lonlat(ipix_max, nside, order = "nested")
print("Highest probability at (RA, dec): ", ra.deg, dec.deg, "with prob [1/deg^2] of: ", skymap[i]['PROBDENSITY'].to_value(u.deg**-2))

Highest probability at (RA, dec):  194.30419921874997 -17.856895095545468 with prob [1/deg^2] of:  0.07825164701914111


Now open the fits file and read basics info from the header

In [16]:
fits_gw = fits.open(path)
header = fits_gw[1].header
dist_mean = header["DISTMEAN"]
dist_std = header["DISTSTD"]
print("The dist_mean and dist_std values: ", dist_mean*u.Mpc, dist_std*u.Mpc)

The dist_mean and dist_std values:  39.76999609489013 Mpc 8.308435058808886 Mpc


### Get DESI data

Establish a connection to the DESI database and load the daily data.

In [18]:
try:
    db = psycopg2.connect(host='decatdb.lbl.gov', database='desidb', user='desi', password = "5kFibers!", port="5432")
    cursor = db.cursor()
except (Exception, psycopg2.Error) as error:
    print(error)

cursor = db.cursor()

connection to server at "decatdb.lbl.gov" (128.3.71.86), port 5432 failed: Operation timed out
	Is the server running on that host and accepting TCP/IP connections?



NameError: name 'db' is not defined

In [ ]:
redux = 'daily'
query = 'SELECT f.targetid,f.target_ra,f.target_dec,c.tileid,c.night,r.z,r.zerr,r.zwarn,r.deltachi2,f.flux_z,f.bgs_target,f.ebv, f.sersic, f.mws_target, c.filename\n' \
                    f'FROM {redux}.tiles_fibermap f\n' \
                    f'INNER JOIN {redux}.cumulative_tiles c ON f.cumultile_id=c.id\n' \
                    f'INNER JOIN {redux}.tiles_redshifts r ON r.cumultile_id=c.id AND r.targetid=f.targetid\n' \
                    f'WHERE q3c_radial_query( f.target_ra, f.target_dec, {ra}, {dec}, 50);'

cursor.execute(query)
rows = cursor.fetchall()

## Do all the calculations, selections,...

- turn the daily DESI data into a Table
- do some basic data selection stuff: only keep good + positive redshifts, only positive values of z-band flux and eliminate duplicates (right now simply takes the first entry)
- add the ipix (healpix) pixel value for each target
- calculate the probability for each target
- calculate the probabilities for each object
- add these values to the able ("PROB"), select only targets within the 90% credible region (i.e. with a certainty of 90% the GW event is inside this region)
- calculate the distances and its errors from the redshift, add to the data table
- calculate the 3D probability just like here: https://iopscience.iop.org/article/10.3847/0067-0049/226/1/10 (§4) and add these values to the table
- sort the table by its 3D probability (descending)
- calculate absolute and apparent magnitude, luminosity and add to table

The final step is to include the calculation of the most likely host galaxy from https://arxiv.org/pdf/1710.05452.pdf (GW170817); these values are then also added to the table as "P_GAL"

In [ ]:
if rows:
    data = Table(list(map(list, zip(*rows))),
                             names=['TARGETID', 'TARGET_RA', 'TARGET_DEC', 'TILEID', 'NIGHT', 'Z', 'ZERR', 'ZWARN', 'DELTACHI2', 'FLUX_Z', 'BGS_TARGET', 'EBV', 'MWS_TARGET','FILENAME'])
data[:5] 

basic data clean up:

In [ ]:
data = data[data['ZWARN']==0]
data = data[data['Z']>=0]
data = data[data['FLUX_Z']>0]
data = table.unique(data, keys = "TARGETID")

calculate the healpix id for every object in the catalog and its probability; then append this data to the Table

In [27]:
max_level = 29 # highest possible HEALPix resolution that can be represented in a 64-bit signed integer
max_nside = ah.level_to_nside(max_level)
level, ipix = ah.uniq_to_level_ipix(skymap["UNIQ"])
index = ipix*(2**(max_level-level))**2

sorter = np.argsort(index)

match_ipix = ah.lonlat_to_healpix(data["RA"], data["DEC"], max_nside, order='nested')

i = sorter[np.searchsorted(index, match_ipix, side='right', sorter=sorter) - 1]

probs = skymap[i]['PROBDENSITY'].to_value(u.deg**-2)

NameError: name 'data' is not defined

In [26]:
ra,dec

(<Longitude 3.39124803 rad>, <Latitude -0.31166161 rad>)

In [ ]:
data = hstack([dat, Table({"UNIQ"})])